In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from datasets import load_dataset

# 1. 加载数据集
dataset = load_dataset("ibrahimhamamci/DENTEX", split="train")  # 使用Huggingface上的DENTEX数据集

# 2. 定义数据处理函数
def preprocess_images(example):
    image = example['image']  # 假设数据集中有'image'字段
    boxes = example['boxes']  # 处理bounding box
    labels = example['labels']  # 处理label (c)字段，异常牙齿的标签
    image = F.to_tensor(image)
    return image, {'boxes': torch.tensor(boxes), 'labels': torch.tensor(labels)}

# 3. 数据预处理
dataset = dataset.map(preprocess_images)

# 4. 加载Faster R-CNN模型
model = fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  # 假设只有两类：正常牙齿和异常牙齿
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# 5. 训练模型
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 定义优化器
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# 训练循环
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in dataset:  # 假设已经被DataLoader封装
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # 前向传播
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # 反向传播和优化
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {losses.item()}")

# 6. 保存模型
torch.save(model.state_dict(), "fasterrcnn_dental_model.pth")

# 7. 模型评估（可以基于AP, AR, F1等指标进行）
# 此处可以加入代码进行测试集的评估。


In [1]:
!pip install ultralytics
!pip install datasets


   ---------------------------------------- 0.0/877.5 kB ? eta -:--:--
   ---------------------------------------- 877.5/877.5 kB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.1 MB 13.4 MB/s eta 0:00:02
   ------- -------------------------------- 5.0/25.1 MB 12.1 MB/s eta 0:00:02
   ------------ --------------------------- 8.1/25.1 MB 13.2 MB/s eta 0:00:02
   ---------------- ----------------------- 10.2/25.1 MB 12.5 MB/s eta 0:00:02
   -------------------- ------------------- 12.8/25.1 MB 12.6 MB/s eta 0:00:01
   ------------------------ --------------- 15.5/25.1 MB 12.6 MB/s eta 0:00:01
   --------------------------- ------------ 17.6/25.1 MB 12.2 MB/s eta 0:00:01
   -------------------------------- ------- 20.2/25.1 MB 12.3 MB/s eta 0:00:01
   ----------------------------------- ---- 22.3/25.1 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------  24.9/25.1 MB 12.1 MB/s eta 

In [2]:
from datasets import load_dataset

# 加载DENTEX数据集
dataset = load_dataset("ibrahimhamamci/DENTEX")

# 查看数据集的结构和样本
print(dataset)
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 3603
    })
    validation: Dataset({
        features: ['image'],
        num_rows: 50
    })
    test: Dataset({
        features: ['image'],
        num_rows: 250
    })
})
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2955x1316 at 0x2084227ED60>}


In [3]:
import os
from PIL import Image

# 定义保存YOLO格式数据的目录
train_dir = 'datasets/DENTEX/train'
val_dir = 'datasets/DENTEX/val'

# 创建YOLO格式的标签文件夹
os.makedirs(f'{train_dir}/labels', exist_ok=True)
os.makedirs(f'{val_dir}/labels', exist_ok=True)
os.makedirs(f'{train_dir}/images', exist_ok=True)
os.makedirs(f'{val_dir}/images', exist_ok=True)

# 定义函数用于将bounding box转换为YOLO格式
def convert_to_yolo_format(image_size, box):
    x_center = (box[0] + box[2]) / 2.0 / image_size[0]
    y_center = (box[1] + box[3]) / 2.0 / image_size[1]
    width = (box[2] - box[0]) / image_size[0]
    height = (box[3] - box[1]) / image_size[1]
    return [x_center, y_center, width, height]

# 处理训练集数据并生成YOLO格式的标签
for i, example in enumerate(dataset['train']):
    image = example['image']
    boxes = example['boxes']  # 假设每个样本有 'boxes' 字段，包含 bounding box 数据
    labels = example['labels']  # 假设有类别标签 'labels'
    
    # 保存图片
    image_path = f'{train_dir}/images/{i}.jpg'
    image.save(image_path)
    
    # 打开对应的标签文件
    label_path = f'{train_dir}/labels/{i}.txt'
    with open(label_path, 'w') as f:
        for label, box in zip(labels, boxes):
            yolo_box = convert_to_yolo_format(image.size, box)
            # 将类别ID和转换后的box写入标签文件
            f.write(f"{label} {' '.join(map(str, yolo_box))}\n")

print("训练集处理完成！")


KeyError: 'boxes'

In [4]:
# 查看数据集中每个样本的字段
print(dataset['train'].features)


{'image': Image(mode=None, decode=True, id=None)}


In [5]:
# 检查数据集的键和分片
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 3603
    })
    validation: Dataset({
        features: ['image'],
        num_rows: 50
    })
    test: Dataset({
        features: ['image'],
        num_rows: 250
    })
})


In [6]:
# 假设你已经加载了DENTEX数据集
# 只选择类型(c)的数据

# 假设dataset['train']中包含不同类型的标注
# 使用过滤条件筛选出类型(c)的数据 (这里需要根据实际字段进行调整)
c_type_data = [example for example in dataset['train'] if example['label_type'] == 'c']

print(f"类型(c)数据量: {len(c_type_data)}")


KeyError: 'label_type'

In [7]:
# 查看第一个样本的完整内容，看看是否有其他字段存储类型信息
print(dataset['train'][0])


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2955x1316 at 0x20842575130>}
